In [1]:
from regressors import *
import pickle
import pandas as pd

In [2]:
ensembleregression = EnsembleRegression(n_regressors=5,n_variables=3, eps=3, decision_boundary=0.6, global_search=False)
spatialcomparision = SpatialComparision()

In [3]:
n_stations = 450
n_timesteps= 100 #maybe 288 is the maximum to propoerly manage them 
k = 5
p_noise_stations = 0.05
p_noise_timesteps= 0.05


data = Tempearture_DWD(n_stations, n_timesteps, k, p_noise_stations=p_noise_stations, p_noise_timesteps= p_noise_timesteps)
validatorNew= Executor(data, ensembleregression)
validatorOld = Executor(data, spatialcomparision)
resultNew = validatorNew.evaluate_validator()  
resultOld = validatorOld.evaluate_validator()  




In [ ]:
plt.imshow(data.matrix_noises)
plt.colorbar()

In [ ]:
def regression_predict_w_randomness(ts_data, i, global_search=False, display=False) :     
    max_score = -np.inf
    max_iteration = 50
    n_iter = 0 
    thr = 0.9
    while True :
        if global_search : 
            candidate = np.arange(ts_data.shape[0])
            candidate = np.delete(candidate, i)                      
        else : 
            candidate = data._neighbor[i]      
        idx_stations = np.random.choice(candidate, size=3, replace=False)
        idx_timesteps = np.random.choice(np.arange(ts_data.shape[1]), size= round(ts_data.shape[1]), replace=True)
        idx_timesteps.sort()

        y = ts_data[i]
        X = ts_data[idx_stations]
        reg = LinearRegression().fit(X.T[idx_timesteps],y[idx_timesteps])
        score = reg.score(X.T[idx_timesteps],y[idx_timesteps])


        if score > max_score :
            max_score = score
            max_idx_stations = idx_stations
            max_idx_timesteps = idx_timesteps

        if n_iter >= max_iteration :                 
            X = ts_data[max_idx_stations]
            reg = LinearRegression().fit(X.T[max_idx_timesteps],y[max_idx_timesteps])
            pred = reg.predict(X.T)
            gap = abs(pred - y)
            if display : 
                print("for {} iteration, cannot find more than {}. the best upto now is {}".format(max_iteration, thr, max_score))
            break
        if (score > thr) : 
            pred = reg.predict(X.T)
            gap = pred - y
            if display : 
                print(score)
            break
        n_iter += 1
    if display : 
        plt.figure(figsize=(20,3))
        plt.plot(X.T, color='grey')
        plt.plot(y)
        plt.plot(pred, color='black')
        # plt.plot(gap, color='blue')
    return pred

In [ ]:
def regression_predict_w_neighbor(ts_data, i) :     
    y = ts_data[i]
    X = ts_data[neighbor[i]]
    reg = LinearRegression().fit(X.T,y)
    score = reg.score(X.T,y)
    print(score)
    pred = reg.predict(X.T)
    gap2 = abs(pred - y)
    plt.figure(figsize=(20,5))
    plt.plot(X.T, color='grey')
    plt.plot(y)
    plt.plot(pred, color='black')
    # plt.plot(gap, color='blue')
    return pred

In [13]:
def evaluate_predictor(function, ts_data, i, global_search=False, display = False) : 
    pred = function(ts_data, i, global_search=global_search, display=display)

    gap = pred - ts_data[i]
    gap_between_rawdata = pred - data._ts_rawdata[i]
    
    gap_mean = np.mean(gap)
    gap_std = max(np.std(gap),2)

    idx = (gap >= gap_mean +  2 * gap_std) | (gap <= gap_mean -  2 * gap_std)
    pred_by_neighbor_randomness = np.where(idx == True )[0]
    
    if display  : 
        print("SSE between rawdata", math.sqrt(sum(gap_between_rawdata**2)))
        print(pred_by_neighbor_randomness)
        print("LevenshteinDistance", levenshteinDistance(answer, pred_by_neighbor_randomness))
        plt.plot(idx, color='red')
        
        
        plt.figure(figsize=(20,3))
        plt.plot(pred, color='black')        
        plt.plot(data._ts_rawdata[i], color='green')
        
    return levenshteinDistance(answer, pred_by_neighbor_randomness)

# Random Selection

In [29]:
n_stations = 450
n_timesteps= 50 #maybe 288 is the maximum to propoerly manage them 
k = 5
p_noise_stations = 0.10
p_noise_timesteps= 0.10
print("# of noises:", n_stations * n_timesteps * p_noise_stations * p_noise_timesteps)
print("# of data :", n_stations * n_timesteps )

data = Tempearture_DWD(n_stations, n_timesteps, k, p_noise_stations=p_noise_stations, p_noise_timesteps= p_noise_timesteps)

# of noises: 225.0
# of data : 22500


In [30]:
ts_data = data.ts_data
neighbor = data._neighbor

In [31]:
correct_and_predict()

55

In [32]:
new_predict(), old_predict(), correct_and_predict()

(309, 1554, 55)

In [ ]:
i = np.random.choice(data._picked_stations)
# i = np.random.choice(range(len(ts_data)))
try : 
    answer = sorted(data.dic_timesteps[i])
except :
    answer = []
    
pred_by_new = [int(s.split('_')[1]) for s in validatorNew.validate(i)]
pred_by_old = [int(s.split('_')[1]) for s in validatorOld.validate(i)]
print("selected station", i)
print("neighbor stations with noise: ", [s for s in neighbor[i] if s in data._picked_stations])
print("answer:         ",answer)
print("predicted by new",pred_by_new, levenshteinDistance(answer, pred_by_new ))
print("predicted by old",pred_by_old, levenshteinDistance(answer, pred_by_old ))

In [7]:
def new_predict() : 
    s=0
    for i in range(len(ts_data)) : 
        try : 
            answer = sorted(data.dic_timesteps[i])
        except : 
            answer = []
        pred_by_new = [int(s.split('_')[1]) for s in validatorNew.validate(i)]
        s += levenshteinDistance(answer, pred_by_new )

    return s

In [8]:
def old_predict() : 
    s=0
    for i in range(len(ts_data)) : 
        try : 
            answer = sorted(data.dic_timesteps[i])
        except : 
            answer = []
        pred_by_old = [int(s.split('_')[1]) for s in validatorOld.validate(i)]
        s += levenshteinDistance(answer, pred_by_old )
    return s


In [9]:
def correct_and_predict() : 
    s=0
    for i in range(len(ts_data)) : 


        try : 
            answer = sorted(data.dic_timesteps[i])
        except :
            answer = []
        y = copy.deepcopy(ts_data[i])
        X = copy.deepcopy(ts_data[neighbor[i]])
        corrected_y = correct(X,y)
        try : 
            a1 = data.dic_timesteps[i]        
        except : 
            a1 = []
        a1.sort()
        a2 =np.where(abs(ts_data[i] - corrected_y)>3)[0]
        a2.sort()

        s+= levenshteinDistance(a1,a2)
    return s

In [ ]:
new_predict
old_predict()
correct_and_predict()

In [ ]:
s=0
for i in range(len(ts_data)) : 
    s +=  evaluate_predictor(regression_predict_w_randomness, ts_data, i, global_search = True, display=False)
s

In [ ]:

pred_idx = evaluate_predictor(regression_predict_w_randomness, ts_data, i, global_search = True, display=True)



# GUT

In [ ]:
ts_data = data.ts_data
neighbor = data._neighbor

In [ ]:
data._picked_stations

In [ ]:
s

In [ ]:
plt.figure(figsize=(20,3))
i = 406
plt.plot(ts_data[i])
plt.plot(ts_data[neighbor[i]].T, color='grey')

In [ ]:
s

In [ ]:
a2

In [10]:
def correct(X, y) : 
    while True :  
        reg = LinearRegression().fit(X.T, y)
#         print(reg.score(X.T, y))
        pred = reg.predict(X.T)
        error = pred - y 
        error_mean = np.mean(error)
        error_std = np.std(error)
        idx_boolean = (error >= error_mean +  max(2 * error_std,3)) | (error <= error_mean -  max(2 * error_std,3))
        idx = np.where(idx_boolean == True )[0]
        y[idx] = pred[idx]
#         print(idx)

#         plt.figure()
#         plt.plot(X.T, color='grey')
#         plt.plot(y, color='blue')
#         plt.plot(pred, color='red')
        
        if len(idx) == 0 :            
            break
    return y

    

In [ ]:
score_idx = np.zeros(ts_data.shape[1])
for j in range(10) : 
    score_idx += evaluate_predictor(regression_predict_w_randomness, ts_data, i, global_search = False, display=False)

In [ ]:
np.where(score_idx > 6)[0]

In [ ]:
(-score_idx).argsort()[:3]

In [ ]:
np.where(gap>1)[0]

In [ ]:
noise = data.ts_data - data._ts_rawdata
plt.figure(figsize=(10,10))
plt.imshow(noise)
plt.colorbar()


In [ ]:
ts_data = data.ts_data 

In [ ]:
stations_w_noises = set([int(x.split('_')[0]) for x in data.lst_station_timestep])

In [ ]:
validator1.evaluate_validator()

In [ ]:
validator3.evaluate_validator()

# stations with no problems 

In [ ]:

# def find_nearest_stations(i, ts_data, neighbor, max_iteration) : 
def find_nearest_stations(i, ts_data,  max_iteration) : 
    iteration = 0
    max_score = -np.inf
    while True  :
        candidate = list(range(len(ts_data)))
        candidate.remove(i)
#         candidate = neighbor[i]
        idx = np.random.choice(candidate, size=3)
    #     print(idx)
        X = ts_data[idx]
        y = ts_data[i]
        reg = LinearRegression().fit(X.T, y)
        score = reg.score(X.T,y)
        if score >= max_score :
            max_score = score 
            max_idx = idx
        if score >= 0.9 : 
#             print(i, score, idx,iteration )
            return max_score, max_idx

        iteration += 1
    #     print(idx, score, iteration)
        if iteration >= max_iteration : 
#             print("cannot find more than .9, the best so far")
#             plt.plot(y)
#             plt.plot(X.T, color='grey')
            return max_score, max_idx
    

In [ ]:
result = np.zeros(len(data.ts_data))
max_iteration = 100
for i in range(len(data.ts_data)) : 
    max_score, _ = find_nearest_stations(i, data.ts_data, max_iteration)
    result[i] = max_score

In [ ]:
df = pickle.load(open('data_dvd_reduced.p','rb'))
df_metadata = pickle.load(open('metadata.p', 'rb'))
metadata = df_metadata.values[:400]
neighbor = create_neighor_list(metadata[:,1:], k=5)
ts_rawdata = df.values[:400]
ts_preprocessed_data = preprocess_data(ts_rawdata)[:,5000:7000]

In [ ]:
ts_data, ANSWER, lst_noises = add_noise(ts_preprocessed_data, 0.10, 3, 10)


In [ ]:
s = random.choice(list(set([k[0] for k,v in lst_noises.items()])))

plt.figure(figsize=(20,5))
plt.plot(ts_data[s], color='red')
plt.plot(ts_data[neighbor[s]].T , color='grey')
plt.title(s)

d = [(k,v) for k,v in sorted(lst_noises.items(), reverse=False, key=lambda x : x[0][0]) if k[0] == s]
print([(k,v) for k,v in sorted(d, reverse=False, key=lambda x : x[0][1])])
result1 = regression_based_outlier_detection(ts_data, neighbor, s, n_regressors, n_variables, eps=eps)
result2 = regression_based_outlier_detection2(ts_data, neighbor, s, n_regressors, n_variables, eps=eps)
result3 = regression_based_outlier_detection2(ts_data, neighbor, s, n_regressors, n_variables, eps=eps)
result4 = spatialcomparision(ts_data, neighbor, s)

print("classified by new idea", result1)
# print("classified by new idea2", result2)
# print("classified by new idea3", result3)
print("classified by old idea", result4)

In [ ]:
%%timeit
spatialcomparision(ts_data, neighbor, station=s)

In [ ]:
%%timeit
regression_based_outlier_detection(ts_data, neighbor, s, n_regressors=n_regressors, n_variables=n_variables, eps=3)


In [ ]:

n_regressors = 7
result = np.empty(len(ts_data[s]))
for i in range(n_regressors) : 
    idx = random.choices(neighbor[s],k=2)
    X = ts_data[idx]
    y = ts_data[s]
    reg = LinearRegression().fit(X.T, y)
    predict = reg.intercept_ + np.dot(X.T, reg.coef_)
    result += predict    
    print(idx, reg.score(X.T, y))
result = result / n_regressors


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(30,10))
ax1 = plt.subplot(131)
ax2 = plt.subplot(132)
ax3 = plt.subplot(133)

ax1.plot(ts_data[s], color='red')
ax1.plot(ts_data[neighbor[s]].T, color='grey')

ax2.plot(y, color='red')
ax2.plot(result, color='grey')


# ylim = [-1000,40]
# ax1.set_ylim(ylim)
# ax2.set_ylim(ylim)

diff = predict-y

ax3.plot(diff, color='red')


In [ ]:
def regression_based_outlier_detection2(ts_data, neighbor, station, n_regressors, n_variables, eps) :
    result = np.array([], dtype='int')
    coeffs = np.zeros((n_regressors, n_variables + 1))
    score = np.zeros(n_regressors)
    predict = np.zeros((n_regressors, ts_data.shape[1]))
    for i in range(n_regressors) : 
        idx = random.choices(neighbor[station],k=n_variables)
        y = ts_data[station] 
        X = ts_data[idx]
        reg = LinearRegression().fit(X.T, y)
        score[i] = reg.score(X.T, y)
        predict[i] = reg.intercept_ + np.dot(X.T, reg.coef_)
        
    
    print(score)
    idx = (-score).argsort()[:-1]
    score = score / score.sum()    
    return np.dot(predict[idx].T, score[idx])
    
    
    
    
    

In [ ]:
regression_based_outlier_detection2(ts_data, neighbor, s, n_regressors=10, n_variables=3, eps=2)


In [ ]:
predict

In [ ]:
score.sum()

In [ ]:
np.dot(predict.T, score)

# random test

In [ ]:
n_variables = 3
idx = random.choices(neighbor[s],k=n_variables)
print(idx)
y = ts_data[s]
X = ts_data[neighbor[s]]
reg = LinearRegression().fit(X.T, y)
predict = reg.intercept_ + np.dot(X.T, reg.coef_)
score = reg.score(X.T,y)
print(score)

In [ ]:
original = ts_rawdata[s]
predict = reg.intercept_ + np.dot(X.T, reg.coef_)
plt.plot(original)
plt.plot(predict)
plt.plot(original-predict)

In [ ]:
mean = np.mean(diff)
sd = np.std(diff)
result_stat = np.where((diff >= mean + 2* sd)  | (diff  <= mean - 2* sd ))[0]

In [ ]:
ANSWER.sort()
ANSWER

In [ ]:
result_stat